## Helpful Links
- Create New [GMail Account](https://accounts.google.com/signup)
- Turn [Allow less secure apps to ON](https://myaccount.google.com/lesssecureapps)
- [Text to HTML](https://wordtohtml.net/)

In [55]:
import config
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from time import strftime
import datetime
import pandas as pd
import pandas_gbq
from google.oauth2 import service_account
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import requests

In [47]:
credentials = service_account.Credentials.from_service_account_file(
    r'/Users/Nicholas/Desktop/GCP/GCP_Keys/spotify-project-287802-e5e6c43e8ecb.json'
)

client_id = config.client_id
client_secret = config.client_secret
username = config.username

## Basic Info

In [44]:
# user name
username = 'Nick'
#current year
current_year = datetime.date.today().year
# current month
current_month = strftime('%B')

## Track Info

In [25]:
# Query GCP BQ for track duration

project_id = "spotify-project-287802"

sql = """
SELECT rpt.*
    , td.*
    , ad.*
FROM `spotify-project-287802.spotify_api.recently_played_tracks` AS rpt
INNER JOIN `spotify-project-287802.spotify_api.track_data` AS td
  ON td.track_id = rpt.track_id
INNER JOIN `spotify-project-287802.spotify_api.artist_data` AS ad
  ON ad.artist_id = rpt.artist_id
WHERE CAST(CAST(rpt.local_time AS TIMESTAMP) AS DATE) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
ORDER BY rpt.local_time DESC
"""
# Load table into df
bq_df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials)

bq_df.head()

Downloading: 100%|██████████| 771/771 [00:00<00:00, 1493.99rows/s]


,track_id,album_id,artist_id,track_name,artist_name,album_name,track_popularity,time_palyed,local_time,danceability,...,track_href,analysis_url,duration_ms,time_signature,track_duration,artist_id_1,artist_followers,artist_genres,artist_name_1,artist_popularity
0,09akcdeCiV2jIl6x9WpmxA,0tF6XGxRVTrKfnC0UGlTAT,4KJ6jujcNPzOyhdNoiNftp,"don't worry, you will",lovelytheband,everything i could never say...,56,2020-11-20T16:16:51.943Z,2020-11-20 10:16:51,0.751,...,https://api.spotify.com/v1/tracks/09akcdeCiV2j...,https://api.spotify.com/v1/audio-analysis/09ak...,197000,4,00:03:17,4KJ6jujcNPzOyhdNoiNftp,273057,indie pop,lovelytheband,68
1,4sNG6zQBmtq7M8aeeKJRMQ,3HJiLDJgWA9Z0MvCxlzHYQ,2D4FOOOtWycb3Aw9nY5n3c,Brazil,Declan McKenna,What Do You Think About the Car?,73,2020-11-20T05:20:11.691Z,2020-11-19 23:20:11,0.687,...,https://api.spotify.com/v1/tracks/4sNG6zQBmtq7...,https://api.spotify.com/v1/audio-analysis/4sNG...,252307,4,00:04:12,2D4FOOOtWycb3Aw9nY5n3c,610634,indie pop,Declan McKenna,71
2,0U3u0HdRQjD8mAMYBak4Ok,1x8SKoPwUTpOfc0w5gBPsk,3IunaFjvNKj98JW89JYv9u,Saw You In A Dream,The Japanese House,Saw You In A Dream,65,2020-11-20T04:35:44.553Z,2020-11-19 22:35:44,0.607,...,https://api.spotify.com/v1/tracks/0U3u0HdRQjD8...,https://api.spotify.com/v1/audio-analysis/0U3u...,208280,4,00:03:28,3IunaFjvNKj98JW89JYv9u,417760,art pop,The Japanese House,67
3,0X5mtNbqxbiTYkwj0CQc2f,3Z62dgkhjWCfLeZ8v9p2Td,0ZxZlO7oWCSYMXhehpyMvE,Crash My Car,COIN,Dreamland,71,2020-11-19T14:44:28.426Z,2020-11-19 08:44:28,0.244,...,https://api.spotify.com/v1/tracks/0X5mtNbqxbiT...,https://api.spotify.com/v1/audio-analysis/0X5m...,188307,5,00:03:08,0ZxZlO7oWCSYMXhehpyMvE,348344,indie pop,COIN,72
4,0X5mtNbqxbiTYkwj0CQc2f,3Z62dgkhjWCfLeZ8v9p2Td,0ZxZlO7oWCSYMXhehpyMvE,Crash My Car,COIN,Dreamland,71,2020-11-19T13:21:54.126Z,2020-11-19 07:21:54,0.244,...,https://api.spotify.com/v1/tracks/0X5mtNbqxbiT...,https://api.spotify.com/v1/audio-analysis/0X5m...,188307,5,00:03:08,0ZxZlO7oWCSYMXhehpyMvE,348344,indie pop,COIN,72


### Track play count sum

In [30]:
sum_duration_ms = bq_df['duration_ms'].sum()
millis = sum_duration_ms
millis = int(millis)

def convert_from_ms(milliseconds): 
    seconds, milliseconds = divmod(milliseconds,1000) 
    minutes, seconds = divmod(seconds, 60) 
    hours, minutes = divmod(minutes, 60) 
    days, hours = divmod(hours, 24) 
    seconds = seconds + milliseconds/1000 
    return days, hours, minutes, seconds 
 
sum_time = convert_from_ms(millis)
days = sum_time[0]
hours = sum_time[1]
minutes = sum_time[2]
seconds = sum_time[3]

### Count of major/minor

In [38]:
mode_check = bq_df.groupby('mode')['track_id'].nunique()
mode_major_count = mode_check[0]
mode_minor_count = mode_check[1]

if mode_minor_count > mode_major_count:
    message1 = "Who hurt you?"
else:
    message1 = "Keep the good vibes going!"

print(message1)

Keep the good vibes going!


### Average popularity

In [41]:
ave_pop_score = round(bq_df['track_popularity'].mean())
ave_pop_score

57.0

In [42]:
if ave_pop_score >= 90:
    message2 = "Do you only listen to singles on the radio?"
elif 70 <= ave_pop_score < 90 :
    message2 = "You must be new to exploring new music."
elif 50 <= ave_pop_score < 70:
    message2 = "A perfect balance of what's hip and what's new."
elif 10 <= ave_pop_score < 50:
    message2 = "You must be a hipster with all this under-the-radar tracks."
elif ave_pop_score < 10:
    message2 = "You're off the grid in your music taste, what are you listening to??"

print(message2)

A perfect balance of what's hip and what's new.


### Playlist Image

In [51]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-read-collaborative, playlist-read-private, playlist-modify-public, playlist-modify-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [60]:
base_url = 'https://api.spotify.com/v1/playlists/79tXL4BD7MnqvpMZtKDho9/images'
#playlist_id = '79tXL4BD7MnqvpMZtKDho9'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
playlist_image = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url, headers=headers)
playlist_image.append(json.loads(r.text))

image = playlist_image[0][1]['url']

'https://mosaic.scdn.co/300/ab67616d0000b2737636e1c9e67eaafc9f49aefdab67616d0000b27395f754318336a07e85ec59bcab67616d0000b273ab2f8973949159695f65df7bab67616d0000b273ccd63af052f7e438c05f6c94'

In [46]:
test_message = f'''
Hello {username},

Here is your Spotify trends for the month of {current_month} {current_year}:
- You have listend to {days} days, {hours} hours, {minutes} minutes, and {round(seconds)} seconds worth of music.
- There were {mode_major_count} Major and {mode_minor_count} Minor tracks played. {message1}
- You're monthly hipster score is {ave_pop_score} : {message2}

<iframe src="https://open.spotify.com/embed/playlist/79tXL4BD7MnqvpMZtKDho9" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>

Keep on listening!

Automated Email by Nick Drake

'''
    
#- Top 5 tracks added to the Python test playlist {current_year}: {tracks}
test_message

'\nHello Nick,\n\nHere is your Spotify trends for the month of November 2020:\n- You have listend to 2 days, 0 hours, 7 minutes, and 8 seconds worth of music.\n- There were 257 Major and 86 Minor tracks played. Keep the good vibes going!\n- You\'re monthly hipster score is 57.0 : A perfect balance of what\'s hip and what\'s new.\n\n<iframe src="https://open.spotify.com/embed/playlist/79tXL4BD7MnqvpMZtKDho9" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>\n\nKeep on listening!\n\nAutomated Email by Nick Drake\n\n'

## Sending Email through Gmail

In [ ]:
sender_email = "SENDER_EMAIL"
receiver_email = "RECEIVER_EMAIL"
password = config.gmail_password

message = MIMEMultipart("alternative")
message["Subject"] = "Spotify Test Email"
message["From"] = sender_email
message["To"] = receiver_email

# Create the plain-text and HTML version of your message
html = f"""<p style="line-height: 1;">Hello {user_name},</p>
<p style="line-height: 1;"><br></p>
<p style="line-height: 1;">Thank you for listing to Spotify!</p>
<p style="line-height: 1;">You have been a member since {member_since}.&nbsp;</p>
<p style="line-height: 1;"><br></p>
<p style="line-height: 1;">Keep listing,</p>
<p style="line-height: 1;">Automated Email by Nick Drake</p>
"""

# Turn these into plain/html MIMEText objects
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
# message.attach(part1)
# message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )